EXTRACTION PROCESS

In [165]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [166]:
#read csv1
df1 = pd.read_csv("D:/Project/etl_python_sql/resources/all_years_o3.csv")
df1.head()

,Date,Country,City,Specie,count,min (ppb),max (ppb),median (ppb)
0,2017-01-01,US,Los Angeles,o3,24,2.0,31.0,10.0
1,2017-01-01,CN,Shanghai,o3,188,1.0,39.0,11.0
2,2017-01-01,US,Manhattan,o3,24,3.0,28.0,16.0
3,2017-01-01,US,Jacksonville,o3,46,2.0,18.0,12.0
4,2017-01-01,CN,Beijing,o3,235,1.0,8.0,2.0


In [167]:
#drop last 3 columns
df1.drop(
    columns=["min (ppb)", "max (ppb)", "median (ppb)"],
    inplace=True
)
df1.head()

,Date,Country,City,Specie,count
0,2017-01-01,US,Los Angeles,o3,24
1,2017-01-01,CN,Shanghai,o3,188
2,2017-01-01,US,Manhattan,o3,24
3,2017-01-01,US,Jacksonville,o3,46
4,2017-01-01,CN,Beijing,o3,235


In [168]:
#rename the count columns
df1.columns = ['Date', 'Country', 'City', 'Specie', 'Count_o3']
df1.head()

,Date,Country,City,Specie,Count_o3
0,2017-01-01,US,Los Angeles,o3,24
1,2017-01-01,CN,Shanghai,o3,188
2,2017-01-01,US,Manhattan,o3,24
3,2017-01-01,US,Jacksonville,o3,46
4,2017-01-01,CN,Beijing,o3,235


In [173]:
df1_normalized = df1.drop(columns=['Specie'])
df1_normalized.columns = df1_normalized.columns.str.lower()

In [179]:
print(df1_normalized)

            date country          city  count_o3
0     2017-01-01      US   Los Angeles        24
1     2017-01-01      CN      Shanghai       188
2     2017-01-01      US     Manhattan        24
3     2017-01-01      US  Jacksonville        46
4     2017-01-01      CN       Beijing       235
...          ...     ...           ...       ...
5025  2020-06-30      US   Los Angeles        24
5026  2020-06-30      CN         Wuhan       382
5027  2020-06-30      US     Manhattan        24
5028  2020-06-30      US  Jacksonville       120
5029  2020-06-30      CN      Shanghai       828

[5030 rows x 4 columns]


In [192]:
#read csv2
df2 = pd.read_csv("D:/Project/etl_python_sql/resources/all_years_pm25.csv")
df2.head()

,Date,Country,City,Specie,count,min (ug/m3),max (ug/m3),median (ug/m3)
0,2017-01-01,IN,New Delhi,pm25,24,54.9,282.7,177.7
1,2017-01-01,CN,Shanghai,pm25,215,12.0,63.2,34.4
2,2017-01-01,US,Jacksonville,pm25,115,3.1,113.6,8.1
3,2017-01-01,US,Los Angeles,pm25,69,3.3,109.7,15.9
4,2017-01-01,US,Manhattan,pm25,96,4.3,23.0,12.1


In [199]:
df2.columns.tolist()

['Date',
 'Country',
 'City',
 'Specie',
 'count_pm25',
 'min (ug/m3)',
 'max (ug/m3)',
 'median (ug/m3)']

In [222]:
print(df2)

            date country          city specie  count_pm25  min (ug/m3)  \
0     2017-01-01      IN     New Delhi   pm25          24         54.9   
1     2017-01-01      CN      Shanghai   pm25         215         12.0   
2     2017-01-01      US  Jacksonville   pm25         115          3.1   
3     2017-01-01      US   Los Angeles   pm25          69          3.3   
4     2017-01-01      US     Manhattan   pm25          96          4.3   
...          ...     ...           ...    ...         ...          ...   
5767  2020-06-30      US     Manhattan   pm25          49          0.2   
5768  2020-06-30      US       Houston   pm25         120         15.4   
5769  2020-06-30      CN         Wuhan   pm25         384          2.1   
5770  2020-06-30      CN       Beijing   pm25         474          2.1   
5771  2020-06-30      US  Jacksonville   pm25         141          2.8   

      max (ug/m3)  median (ug/m3)  
0           282.7           177.7  
1            63.2            34.4  
2  

In [245]:
# Hapus kolom Specie dari df1 jika ada
if 'Specie' in df1.columns:
    df1 = df1.drop(columns=['Specie'])

In [246]:
# Pastikan nama kolom count_o3 sudah benar
if 'count_03' in df1.columns:
    df1 = df1.rename(columns={'count_03': 'Count_O3'})

In [247]:
# Hapus kolom Specie, min, max, median dari df2
cols_to_drop = ['Specie', 'Min (Ug/M3)', 'Max (Ug/M3)', 'Median (Ug/M3)']
df2 = df2.drop(columns=[col for col in cols_to_drop if col in df2.columns])

In [248]:
# Pastikan nama kolom count_pm25 sudah benar
if 'count_pm25' in df2.columns:
    df2 = df2.rename(columns={'count_pm25': 'Count_Pm25'})

In [249]:
# Ubah semua nama kolom ke format yang konsisten
df1.columns = df1.columns.str.title()  # Date, Country, City, Count_O3
df2.columns = df2.columns.str.title()  # Date, Country, City, Count_Pm25

In [250]:
# Pastikan nama kolom kunci sama
print("Kolom df1:", df1.columns.tolist())
print("Kolom df2:", df2.columns.tolist())

Kolom df1: ['Date', 'Country', 'City', 'Count_O3']
Kolom df2: ['Date', 'Country', 'City', 'Count_Pm25']


In [251]:
# Merge dengan inner/left join
new_df = pd.merge(df1, df2, on=['Date', 'Country', 'City'], how='left')

print("Hasil merge setelah cleaning:")
print(new_df.head())

Hasil merge setelah cleaning:
         Date Country          City  Count_O3  Count_Pm25
0  2017-01-01      US   Los Angeles        24          69
1  2017-01-01      CN      Shanghai       188         215
2  2017-01-01      US     Manhattan        24          96
3  2017-01-01      US  Jacksonville        46         115
4  2017-01-01      CN       Beijing       235         386
